# Parsing Tabular Data

**Originally Contributed by**: Clayton Barrows

## Introduction

An example of how to parse tabular files (CSV) files similar to the format established in
the [RTS-GMLC](github.com/gridmod/rts-gmlc/RTS_Data/SourceData) and create a `System` using
[PowerSystems.jl](github.com/NREL-SIIP/PowerSystems.jl)

### Dependencies

In [1]:
using PowerSystems
using TimeSeries
using Dates

### Fetch Data
PowerSystems.jl links to some test data that is suitable for this example.
Let's download the test data

In [2]:
PowerSystems.download(PowerSystems.TestData; branch = "master") # *note* add `force=true` to get a fresh copy
base_dir = dirname(dirname(pathof(PowerSystems)));

### The tabular data format relies on a folder containing `*.csv` files and a `user_descriptors.yaml` file
First, we'll read the tabular data

In [3]:
RTS_GMLC_DIR = joinpath(base_dir, "data", "RTS_GMLC");
rawsys = PowerSystems.PowerSystemTableData(
    RTS_GMLC_DIR,
    100.0,
    joinpath(RTS_GMLC_DIR, "user_descriptors.yaml"),
    timeseries_metadata_file = joinpath(RTS_GMLC_DIR, "timeseries_pointers.json"),
    generator_mapping_file = joinpath(RTS_GMLC_DIR, "generator_mapping_multi_start.yaml"),
)

[ Info: Parsing csv files in Reserves ...
[ Info: Parsing csv data in DAY_AHEAD_regional_Flex_Down.csv ...
[ Info: Parsing csv data in DAY_AHEAD_regional_Flex_Up.csv ...
[ Info: Parsing csv data in DAY_AHEAD_regional_Reg_Down.csv ...
[ Info: Parsing csv data in DAY_AHEAD_regional_Reg_Up.csv ...
[ Info: Parsing csv data in DAY_AHEAD_regional_Spin_Up_R1.csv ...
[ Info: Parsing csv data in DAY_AHEAD_regional_Spin_Up_R2.csv ...
[ Info: Parsing csv data in DAY_AHEAD_regional_Spin_Up_R3.csv ...
[ Info: Parsing csv data in REAL_TIME_regional_Reg_Down.csv ...
[ Info: Parsing csv data in REAL_TIME_regional_Reg_Up.csv ...
[ Info: Parsing csv data in REAL_TIME_regional_Spin_Up_R1.csv ...
[ Info: Parsing csv data in REAL_TIME_regional_Spin_Up_R2.csv ...
[ Info: Parsing csv data in REAL_TIME_regional_Spin_Up_R3.csv ...
[ Info: Successfully parsed Reserves
[ Info: Parsing csv data in branch.csv ...
[ Info: Successfully parsed branch.csv
[ Info: Parsing csv data in bus.csv ...
[ Info: Successfully pa

PowerSystems.PowerSystemTableData:
  directory:  /Users/pchanpiw/.julia/packages/PowerSystems/61h6O/data/RTS_GMLC
  timeseries_metadata_file:  /Users/pchanpiw/.julia/packages/PowerSystems/61h6O/data/RTS_GMLC/timeseries_pointers.json
  base_power:  100.0
  PowerSystems.InputCategoryModule.InputCategory.BRANCH = 1:  120×14 DataFrame
  PowerSystems.InputCategoryModule.InputCategory.RESERVE = 6:  7×7 DataFrame
  PowerSystems.InputCategoryModule.InputCategory.DC_BRANCH = 3:  1×60 DataFrame
  PowerSystems.InputCategoryModule.InputCategory.BUS = 2:  73×15 DataFrame
  PowerSystems.InputCategoryModule.InputCategory.STORAGE = 8:  22×8 DataFrame
  PowerSystems.InputCategoryModule.InputCategory.GENERATOR = 4:  158×58 DataFrame


### Create a `System`
Next, we'll create a `System` from the `rawsys` data. Since a `System` is predicated on a
time series resolution and the `rawsys` data includes both 5-minute and 1-hour resolution
time series, we also need to specify which time series we want to include in the `System`.
The `time_series_resolution` kwarg filters to only include time series with a matching resolution.

In [4]:
sys = System(rawsys; time_series_resolution = Dates.Hour(1));
horizon = 24;
interval = Dates.Hour(24);
transform_single_time_series!(sys, horizon, interval);
sys

[ Info: Unit System changed to InfrastructureSystems.UnitSystemModule.UnitSystem.DEVICE_BASE = 1
┌ Warning: Missing PowerSystems.InputCategoryModule.InputCategory.LOAD = 5 data.
└ @ PowerSystems ~/.julia/packages/PowerSystems/61h6O/src/parsers/power_system_table_data.jl:213
┌ Warning: Rate 175.0 MW for A1 is larger than the max expected in the range of (min = 47.0, max = 52.0).
└ @ PowerSystems ~/.julia/packages/PowerSystems/61h6O/src/utils/IO/branchdata_checks.jl:148
┌ Warning: Rate 175.0 MW for A2 is larger than the max expected in the range of (min = 47.0, max = 52.0).
└ @ PowerSystems ~/.julia/packages/PowerSystems/61h6O/src/utils/IO/branchdata_checks.jl:148
┌ Warning: Rate 175.0 MW for A3 is larger than the max expected in the range of (min = 47.0, max = 52.0).
└ @ PowerSystems ~/.julia/packages/PowerSystems/61h6O/src/utils/IO/branchdata_checks.jl:148
┌ Warning: Rate 175.0 MW for A4 is larger than the max expected in the range of (min = 47.0, max = 52.0).
└ @ PowerSystems ~/.julia

,ConcreteType,SuperTypes,Count
,String,String,Int64
1,Arc,Topology <: Component <: InfrastructureSystemsComponent <: InfrastructureSystemsType <: Any,109
2,Area,AggregationTopology <: Topology <: Component <: InfrastructureSystemsComponent <: InfrastructureSystemsType <: Any,3
3,Bus,Topology <: Component <: InfrastructureSystemsComponent <: InfrastructureSystemsType <: Any,73
4,GenericBattery,Storage <: StaticInjection <: Device <: Component <: InfrastructureSystemsComponent <: InfrastructureSystemsType <: Any,1
5,HVDCLine,DCBranch <: Branch <: Device <: Component <: InfrastructureSystemsComponent <: InfrastructureSystemsType <: Any,1
6,HydroDispatch,HydroGen <: Generator <: StaticInjection <: Device <: Component <: InfrastructureSystemsComponent <: InfrastructureSystemsType <: Any,20
7,Line,ACBranch <: Branch <: Device <: Component <: InfrastructureSystemsComponent <: InfrastructureSystemsType <: Any,105
8,LoadZone,AggregationTopology <: Topology <: Component <: InfrastructureSystemsComponent <: InfrastructureSystemsType <: Any,3
9,PowerLoad,StaticLoad <: ElectricLoad <: StaticInjection <: Device <: Component <: InfrastructureSystemsComponent <: InfrastructureSystemsType <: Any,51


---

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*